In [1]:
# ==== TF-IDF cosine: query_text ↔ item_description ====
import os
import polars as pl
import numpy as np
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

BASE_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, "Data")
TRAIN_PATH = os.path.join(DATA_DIR, "train-dset.parquet")
TEST_PATH  = os.path.join(DATA_DIR, "test-dset-small.parquet")

TRAIN_TFIDF_DESC = os.path.join(DATA_DIR, "train_tfidf_desc_cos.parquet")
TEST_TFIDF_DESC  = os.path.join(DATA_DIR, "test_tfidf_desc_cos.parquet")


C:\Users\idine\PycharmProjects\Avito_Test\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 1) Уникальные тексты
train_lf = pl.scan_parquet(TRAIN_PATH)
test_lf  = pl.scan_parquet(TEST_PATH)

queries = (pl.concat([train_lf.select("query_id","query_text"),
                      test_lf.select("query_id","query_text")])
           .unique(subset=["query_id"])
           .collect(streaming=True))

items_desc = (pl.concat([train_lf.select("item_id","item_description"),
                         test_lf.select("item_id","item_description")])
              .unique(subset=["item_id"])
              .collect(streaming=True))

q_texts = queries.get_column("query_text").fill_null("").to_list()
d_texts = items_desc.get_column("item_description").fill_null("").to_list()

# (быстро и устойчиво для RU/опечаток)
vec = TfidfVectorizer(analyzer="char_wb", ngram_range=(3,5),
                      min_df=3, max_features=200_000,
                      norm="l2", use_idf=True, sublinear_tf=True)


C:\Users\idine\AppData\Local\Temp\ipykernel_32640\2182524949.py:5: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  queries = (pl.concat([train_lf.select("query_id","query_text"),
C:\Users\idine\AppData\Local\Temp\ipykernel_32640\2182524949.py:10: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  items_desc = (pl.concat([train_lf.select("item_id","item_description"),


In [ ]:
corpus = q_texts + d_texts
tfidf = vec.fit_transform(corpus)   # csr
nq = len(q_texts)
Q = tfidf[:nq]        # queries
D = tfidf[nq:]        # descriptions

In [ ]:
qid2i = {int(q): i for i,q in enumerate(queries["query_id"].to_list())}
iid2j = {int(i): j for j,i in enumerate(items_desc["item_id"].to_list())}
